# Multi-Layer Recurrent Neural Network (RNN)

This implementation defines a **multi-layer RNN**.  
The network consists of:

- An **input layer** of size $d_\text{in}$.
- A sequence of **hidden recurrent layers** with dimensions specified by a list  
  $$
  [h_1, h_2, \dots, h_L]
  $$
  where each $h_\ell$ is the number of hidden units in layer $\ell$.
- An **output layer** of size $d_\text{out}$.

---

## Forward Dynamics

At each time step $t$, the network receives an input vector  
$$
x_t \in \mathbb{R}^{d_\text{in} \times 1}.
$$

Each hidden layer $\ell$ maintains its own hidden state $h_t^{(\ell)}$.  
The update rule for layer $\ell$ is:

$$
h_t^{(\ell)} = \tanh\!\left( W_{xh}^{(\ell)} z_t^{(\ell)} + W_{hh}^{(\ell)} h_{t-1}^{(\ell)} + b_h^{(\ell)} \right),
$$

where:
- $W_{xh}^{(\ell)} \in \mathbb{R}^{h_\ell \times d_\ell}$ are the input weights for layer $\ell$,
- $W_{hh}^{(\ell)} \in \mathbb{R}^{h_\ell \times h_\ell}$ are the recurrent weights between timesteps layer $\ell$,
- $b_h^{(\ell)} \in \mathbb{R}^{h_\ell \times 1}$ is the bias,
- $z_t^{(\ell)}$ is the input to this layer:
  - For the first layer, $z_t^{(1)} = x_t$,
  - For subsequent layers, $z_t^{(\ell)} = h_t^{(\ell-1)}$.

---

## Output

The output at time step $t$ is computed from the last hidden layer $h_t^{(L)}$:

$$
y_t = W_{hy} h_t^{(L)} + b_y,
$$

where:
- $W_{hy} \in \mathbb{R}^{d_\text{out} \times h_L}$ are the output weights,
- $b_y \in \mathbb{R}^{d_\text{out} \times 1}$ is the output bias.

---

## Sequence Processing

Given a sequence of inputs $\{x_1, x_2, \dots, x_T\}$:

1. Initialize all hidden states as zeros:
   $$
   h_0^{(\ell)} = 0 \quad \text{for all } \ell = 1, \dots, L.
   $$
2. For each time step $t = 1, \dots, T$:
   - Update hidden states layer by layer using the recurrence above.
   - Compute the output $y_t$.


---

## Loss Function

We use the **Mean Squared Error (MSE)** loss between the predicted outputs 
$\{y_t\}$ and target outputs $\{ \hat{y}_t \}$ across the sequence:

$$
\mathcal{L} = \frac{1}{T} \sum_{t=1}^{T} \| y_t - \hat{y}_t \|^2,
$$

where:
- $T$ is the sequence length,
- $y_t \in \mathbb{R}^{d_\text{out}}$ is the network prediction at time $t$,
- $\hat{y}_t \in \mathbb{R}^{d_\text{out}}$ is the target at time $t$.

The gradient of the loss w.r.t. the output at time step $t$ is:

$$
\frac{\partial \mathcal{L}}{\partial y_t} = \frac{2}{T} \big( y_t - \hat{y}_t \big).
$$

---

## Backpropagation Through Time (BPTT)

During training, gradients are propagated **backwards through time** 
and **down through the layers**.

---

### Step 1: Output layer gradients

For the output layer:

$$
y_t = W_{hy} h_t^{(L)} + b_y,
$$

the gradients are:

$$
\frac{\partial \mathcal{L}}{\partial W_{hy}} = \sum_{t=1}^T \frac{\partial \mathcal{L}}{\partial y_t} \, h_t^{(L)\top},
$$

$$
\frac{\partial \mathcal{L}}{\partial b_y} = \sum_{t=1}^T \frac{\partial \mathcal{L}}{\partial y_t},
$$

and the gradient w.r.t. the last hidden state is:

$$
\delta_t^{(L)} = W_{hy}^\top \frac{\partial \mathcal{L}}{\partial y_t}.
$$

---

### Step 2: Hidden layer gradients

For each hidden layer $\ell$ at time step $t$:

$$
h_t^{(\ell)} = \tanh\!\left( W_{xh}^{(\ell)} z_t^{(\ell)} + W_{hh}^{(\ell)} h_{t-1}^{(\ell)} + b_h^{(\ell)} \right),
$$

with $z_t^{(1)} = x_t$ and $z_t^{(\ell)} = h_t^{(\ell-1)}$ for deeper layers.

The local derivative of the activation is:

$$
\frac{\partial h_t^{(\ell)}}{\partial a_t^{(\ell)}} = 1 - \big(h_t^{(\ell)}\big)^2,
$$

where $a_t^{(\ell)}$ is the pre-activation input to the $\tanh$.

The backpropagated error signal is:

$$
\delta_t^{(\ell)} = \left( \delta_t^{(\ell)} + \delta_{t+1}^{(\ell)} \cdot W_{hh}^{(\ell)\top} \right) \odot \left(1 - \big(h_t^{(\ell)}\big)^2\right),
$$

where:
- $\odot$ is elementwise multiplication,
- $\delta_{t+1}^{(\ell)}$ carries gradient from the next timestep,
- $\delta_t^{(\ell)}$ carries gradient from higher layers at the same timestep.

---

### Step 3: Parameter gradients for hidden layers

The gradients of parameters for each layer $\ell$ are:

$$
\frac{\partial \mathcal{L}}{\partial W_{xh}^{(\ell)}} = \sum_{t=1}^T \delta_t^{(\ell)} \, z_t^{(\ell)\top},
$$

$$
\frac{\partial \mathcal{L}}{\partial W_{hh}^{(\ell)}} = \sum_{t=1}^T \delta_t^{(\ell)} \, h_{t-1}^{(\ell)\top},
$$

$$
\frac{\partial \mathcal{L}}{\partial b_h^{(\ell)}} = \sum_{t=1}^T \delta_t^{(\ell)}.
$$

---

### Step 4: Gradient clipping and updates

To prevent exploding gradients, we apply **clipping**:

$$
g \leftarrow \max\!\left(-c, \, \min(g, c)\right),
$$

for some threshold $c$.

Finally, parameters are updated with **stochastic gradient descent (SGD)**:

$$
\theta \leftarrow \theta - \eta \, \frac{\partial \mathcal{L}}{\partial \theta},
$$

where $\eta$ is the learning rate.

---

## Summary

- Loss is computed as sequence-wide MSE.  
- Gradients are propagated backward through both **time** (via recurrent connections) and **layers** (via depth).  
- Parameter updates use SGD with gradient clipping.  

This procedure allows the RNN to learn temporal dependencies in sequential data.



In [1]:
import numpy as np

class MultiLayerRNN:
    def __init__(self, input_size, hidden_sizes, output_size, seed=42):
        np.random.seed(seed)
        self.hidden_sizes = hidden_sizes
        self.num_layers = len(hidden_sizes)

        # Store weight matrices for each hidden layer
        self.Wxh = []  # input in same timstep → hidden in same timstep
        self.Whh = []  # hidden in prev timestep → hidden current timestep
        self.bh = []   # biases

        prev_size = input_size
        for h_size in hidden_sizes:
            self.Wxh.append(np.random.randn(h_size, prev_size) * 0.01)
            self.Whh.append(np.random.randn(h_size, h_size) * 0.01)
            self.bh.append(np.zeros((h_size, 1)))
            prev_size = h_size

        # Output layer
        self.Why = np.random.randn(output_size, hidden_sizes[-1]) * 0.01
        self.by = np.zeros((output_size, 1))

        # For convenience, put weights in lists of np arrays
        self.Wxh = np.array(self.Wxh, dtype=object)
        self.Whh = np.array(self.Whh, dtype=object)
        self.bh = np.array(self.bh, dtype=object)

    def forward(self, inputs):
        """
        inputs: list of column vectors (input_size × 1)
        Returns: outputs, caches for BPTT
        """
        hs = [np.zeros((h, 1)) for h in self.hidden_sizes]
        outputs, cache = [], []

        for x in inputs:
            layer_input = x
            new_hs = []
            for l in range(self.num_layers):
                h_prev = hs[l]
                h = np.tanh(
                    np.dot(self.Wxh[l], layer_input) +
                    np.dot(self.Whh[l], h_prev) +
                    self.bh[l]
                )
                new_hs.append(h)
                layer_input = h
            hs = new_hs
            y = np.dot(self.Why, hs[-1]) + self.by
            outputs.append(y)

            cache.append((x, hs.copy()))  # store for BPTT

        return outputs, cache

    def compute_loss(self, outputs, targets):
        """
        Mean squared error loss
        """
        loss = 0.0
        for y, t in zip(outputs, targets):
            loss += np.sum((y - t) ** 2)
        return loss / len(outputs)

    def backward(self, outputs, targets, cache, learning_rate=1e-2):
        """
        Backpropagation Through Time (BPTT)
        """
        # Gradients init
        dWxh = [np.zeros_like(W) for W in self.Wxh]
        dWhh = [np.zeros_like(W) for W in self.Whh]
        dbh = [np.zeros_like(b) for b in self.bh]
        dWhy = np.zeros_like(self.Why)
        dby = np.zeros_like(self.by)

        # Initialize hidden state gradients
        dh_next = [np.zeros((h, 1)) for h in self.hidden_sizes]

        # Backward through time
        for t in reversed(range(len(outputs))):
            y, target = outputs[t], targets[t]
            dy = 2.0 * (y - target)  # dL/dy
            dWhy += np.dot(dy, cache[t][1][-1].T)
            dby += dy

            dh = np.dot(self.Why.T, dy)  # gradient into last hidden layer

            for l in reversed(range(self.num_layers)):
                h = cache[t][1][l]
                h_prev = np.zeros_like(h) if t == 0 else cache[t-1][1][l]

                # Add gradient flowing from above in multi-layer case
                dh += dh_next[l]

                # Backprop through tanh: derivative (1 - h^2)
                dtanh = (1 - h * h) * dh

                # Gradients
                dWxh[l] += np.dot(dtanh, cache[t][0].T if l == 0 else cache[t][1][l-1].T)
                dWhh[l] += np.dot(dtanh, h_prev.T)
                dbh[l] += dtanh

                # Propagate gradient backwards in time and to lower layers
                dh_next[l] = np.dot(self.Whh[l].T, dtanh)
                dh = np.dot(self.Wxh[l].T, dtanh)  # pass to lower layer

        # Clip gradients to prevent exploding
        for dparam in dWxh + dWhh + dbh + [dWhy, dby]:
            np.clip(dparam, -5, 5, out=dparam)

        # SGD update
        for l in range(self.num_layers):
            self.Wxh[l] -= learning_rate * dWxh[l]
            self.Whh[l] -= learning_rate * dWhh[l]
            self.bh[l]  -= learning_rate * dbh[l]
        self.Why -= learning_rate * dWhy
        self.by  -= learning_rate * dby




In [8]:

# --- Example usage ---
if __name__ == "__main__":
    np.random.seed(0)
    input_size = 2
    hidden_sizes = [4, 6, 4]
    output_size = 2
    seq_length = 10

    rnn = MultiLayerRNN(input_size, hidden_sizes, output_size)

    # Dummy training data: try to learn identity mapping (predict next x = input)
    for epoch in range(1000):
        # Random input sequence
        inputs = [np.random.randn(input_size, 1) for _ in range(seq_length)]
        targets = inputs  # try to copy input

        outputs, cache = rnn.forward(inputs)
        loss = rnn.compute_loss(outputs, targets)
        rnn.backward(outputs, targets, cache, learning_rate=1e-2)

        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")

Epoch 0, Loss: 1.6369
Epoch 20, Loss: 1.5723
Epoch 40, Loss: 2.2093
Epoch 60, Loss: 2.2362
Epoch 80, Loss: 2.1613
Epoch 100, Loss: 1.8064
Epoch 120, Loss: 2.6874
Epoch 140, Loss: 2.4344
Epoch 160, Loss: 2.1908
Epoch 180, Loss: 2.0478
Epoch 200, Loss: 1.8467
Epoch 220, Loss: 1.4672
Epoch 240, Loss: 1.8439
Epoch 260, Loss: 2.0002
Epoch 280, Loss: 1.2825
Epoch 300, Loss: 1.4901
Epoch 320, Loss: 2.2894
Epoch 340, Loss: 2.2067
Epoch 360, Loss: 2.1660
Epoch 380, Loss: 1.7329
Epoch 400, Loss: 1.7152
Epoch 420, Loss: 1.8073
Epoch 440, Loss: 1.6523
Epoch 460, Loss: 0.8341
Epoch 480, Loss: 1.4956
Epoch 500, Loss: 1.7462
Epoch 520, Loss: 2.8333
Epoch 540, Loss: 2.3198
Epoch 560, Loss: 2.3375
Epoch 580, Loss: 2.2884
Epoch 600, Loss: 1.9303
Epoch 620, Loss: 1.3328
Epoch 640, Loss: 2.4850
Epoch 660, Loss: 2.1811
Epoch 680, Loss: 2.6325
Epoch 700, Loss: 2.4115
Epoch 720, Loss: 1.5769
Epoch 740, Loss: 1.5314
Epoch 760, Loss: 2.5571
Epoch 780, Loss: 2.3426
Epoch 800, Loss: 1.9860
Epoch 820, Loss: 2.872

In [7]:
rnn.forward(np.array(list(range(seq_length))).reshape(-1,2)/seq_length)


([array([[ 0.09457819,  0.0945782 ,  0.09457819,  0.09457819],
         [-0.09716841, -0.09716842, -0.09716841, -0.09716842]]),
  array([[ 0.09456884,  0.09456888,  0.09456883,  0.09456888],
         [-0.09716798, -0.097168  , -0.09716797, -0.097168  ]]),
  array([[ 0.09456858,  0.09456864,  0.09456855,  0.09456864],
         [-0.09716798, -0.09716801, -0.09716797, -0.09716801]]),
  array([[ 0.09456857,  0.09456866,  0.09456854,  0.09456867],
         [-0.09716798, -0.09716802, -0.09716797, -0.09716803]]),
  array([[ 0.09456858,  0.0945687 ,  0.09456853,  0.0945687 ],
         [-0.09716799, -0.09716804, -0.09716797, -0.09716804]])],
 [(array([0. , 0.1]),
   [array([[-0.00013748,  0.00151443, -0.00023568,  0.00075999],
           [-0.0001385 ,  0.00151342, -0.0002367 ,  0.00075897],
           [-0.00013656,  0.00151536, -0.00023475,  0.00076092],
           [-0.00013802,  0.00151389, -0.00023622,  0.00075945]]),
    array([[ 9.90486207e-06, -1.01750061e-05,  1.10984650e-05,
            